This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [1]:
structure_name = "LLZO_010_O_order5_off__Li_110_slab_heavy"

In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_O_order5_off__Li_110_slab_heavy
- Initial lattice height: 71.85 Å
- Number of atoms: 842


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 22.63 Å
Lowest Li slab atom: 34.22 Å
LLZO top z: 34.21 Å
Li penetration into LLZO: -0.01 Å


In [4]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 19.21 Å
Li slab thickness:   22.63 Å
LLZO top z:          34.21 Å
Lowest Li atom z:    34.22 Å
Li penetration into LLZO: -0.01 Å


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 636 atoms out of 842
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_175549/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 20:51:00    -1949.906570        3.904140
SciPyFminCG:    1 20:51:04    -1996.707457        5.475148
SciPyFminCG:    2 20:51:08    -2098.390157        8.169750
SciPyFminCG:    3 20:51:10    -2185.039007        3.123722
SciPyFminCG:    4 20:51:13    -2200.514905        3.430855
SciPyFminCG:    5 20:51:15    -2203.158561        2.002467
SciPyFminCG:    6 20:51:18    -2207.089015        1.659509
SciPyFminCG:    7 20:51:20    -2209.961725        1.495383
SciPyFminCG:    8 20:51:22    -2213.401751        1.387777
SciPyFminCG:    9 20:51:24    -2215.719191        1.385055
SciPyFminCG:   10 20:51:26    -2217.280612        1.877819
SciPyFminCG:   11 20:51:28    -2219.633384        1.906680
SciPyFminCG:   12 20:51:30    -2221.706111        2.654606
SciPyFminCG:   13 20:51:32    -2223.906114        1.691359
SciPyFminCG:   14 20:51:34    -2226.048100        3.265497
SciPyFminCG:   15 20:51:36    -2228.238064        1.726318

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_175549/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 20:57:47    -2259.100126        0.124850
FIRE:    1 20:57:48    -2259.100528        0.063184
FIRE:    2 20:57:49    -2259.101732        0.074450
FIRE:    3 20:57:50    -2259.102134        0.099953
FIRE:    4 20:57:51    -2259.103539        0.080351
FIRE:    5 20:57:52    -2259.104141        0.059828
FIRE:    6 20:57:53    -2259.105346        0.081979
FIRE:    7 20:57:54    -2259.106149        0.049537


In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 22.39 Å
Lowest Li slab atom: 34.46 Å
LLZO top z: 33.84 Å
Li penetration into LLZO: -0.62 Å


In [10]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)

             Step     Time          Energy          fmax
SciPyFminCG:    0 20:57:56    -2259.106149        3.949832
SciPyFminCG:    1 20:58:01    -2388.129382        8.104365
SciPyFminCG:    2 20:58:02    -2472.115504        2.708677
SciPyFminCG:    3 20:58:06    -2526.615093        5.898533
SciPyFminCG:    4 20:58:06    -2532.515893        3.716974
SciPyFminCG:    5 20:58:08    -2541.176984        1.602665
SciPyFminCG:    6 20:58:11    -2545.358172        2.127158
SciPyFminCG:    7 20:58:13    -2548.305762        1.086239
SciPyFminCG:    8 20:58:15    -2550.619588        3.268270
SciPyFminCG:    9 20:58:17    -2554.514309        1.172537
SciPyFminCG:   10 20:58:19    -2556.692229        1.390015
SciPyFminCG:   11 20:58:21    -2558.968716        2.021119
SciPyFminCG:   12 20:58:22    -2562.791569        1.233201
SciPyFminCG:   13 20:58:24    -2568.500654        2.190122
SciPyFminCG:   14 20:58:25    -2571.229628        2.040364
SciPyFminCG:   15 20:58:27    -2575.209064        1.183284

In [11]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_175549/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 21:07:03    -2955.890747        0.196524
FIRE:    1 21:07:03    -2955.893156        0.196461


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.6769436736622497e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:    2 21:07:04    -2955.897573        0.196340


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.748425776693973e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


...

FIRE:  111 21:07:52    -2958.674927        0.051631


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.735299817702863e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  112 21:07:53    -2958.684362        0.050291


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.639969520929463e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  113 21:07:53    -2958.694400        0.048952


In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_010_O_order5_off__Li_110_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 